In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
import sklearn

import torch                                       #Pytorch pkg & tensor library
import torchvision          

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import joblib

In [2]:
data_mix_out= pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data_mix_out.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Seperating 1s and 0s outputs from data

In [3]:
# Seperating 1s and 0s outputs from data
ones=[]
zeros=[]
for i in range(len(data_mix_out)):
    if data_mix_out["Outcome"][i]==1:
        ones.append(i) #(i) because builtin_function_or_method' object is not subscriptable
        
    elif data_mix_out["Outcome"][i]==0:
        zeros.append(i) #(i) because builtin_function_or_method' object is not subscriptable
        
# Defining a function to make dataframe with speific index of data        
def sep_data(data_list):
    combined_csv = pd.concat([pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv', skiprows= i+1, nrows=1, header=None) for i in data_list])
    combined_csv= combined_csv.set_index(i for i in range(len(data_list)))
    combined_csv.columns = data_mix_out.columns
    return combined_csv

# Data containing only 1s output
data_1s= sep_data(ones)

# Data containing only 0s output
data_0s= sep_data(zeros)

# Displaying datas
print("Data with only 1s outcome:")
display(data_1s)

print("Data with only 0s outcome:")
display(data_0s)

Data with only 1s outcome:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,8,183,64,0,0,23.3,0.672,32,1
2,0,137,40,35,168,43.1,2.288,33,1
3,3,78,50,32,88,31.0,0.248,26,1
4,2,197,70,45,543,30.5,0.158,53,1
...,...,...,...,...,...,...,...,...,...
263,1,128,88,39,110,36.5,1.057,37,1
264,0,123,72,0,0,36.3,0.258,52,1
265,6,190,92,0,0,35.5,0.278,66,1
266,9,170,74,31,0,44.0,0.403,43,1


Data with only 0s outcome:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,1,89,66,23,94,28.1,0.167,21,0
2,5,116,74,0,0,25.6,0.201,30,0
3,10,115,0,0,0,35.3,0.134,29,0
4,4,110,92,0,0,37.6,0.191,30,0
...,...,...,...,...,...,...,...,...,...
495,9,89,62,0,0,22.5,0.142,33,0
496,10,101,76,48,180,32.9,0.171,63,0
497,2,122,70,27,0,36.8,0.340,27,0
498,5,121,72,23,112,26.2,0.245,30,0


## Joining the above dataframes 

In [4]:
data= pd.concat([data_0s, data_1s])
data= data.set_index(i for i in range(len(data_0s) + len(data_1s)))
display(data.head())
data.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,1,89,66,23,94,28.1,0.167,21,0
2,5,116,74,0,0,25.6,0.201,30,0
3,10,115,0,0,0,35.3,0.134,29,0
4,4,110,92,0,0,37.6,0.191,30,0


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,1,128,88,39,110,36.5,1.057,37,1
764,0,123,72,0,0,36.3,0.258,52,1
765,6,190,92,0,0,35.5,0.278,66,1
766,9,170,74,31,0,44.0,0.403,43,1
767,1,126,60,0,0,30.1,0.349,47,1


### In the above data, first 500 rows are with outcome '0' and last 268 are with outcome '1'   

In [5]:
# pp.ProfileReport(data)

The Zero values in 	[Glucose, BloodPressure, SkinThickness, Insulin, BMI] can be due to missing data because its impossible to have those values '0' practically

In [6]:
# Not including pregnancies
cols= ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]
k=0

for j in range(len(cols)-1):
    for i in range(len(data)):
        if data[cols[j+1]][i]==0:
            k=k+1
    print('Missing values in  ' + cols[j+1]+'='+ str(k))
    k=0

Missing values in  Glucose=5
Missing values in  BloodPressure=35
Missing values in  SkinThickness=227
Missing values in  Insulin=374
Missing values in  BMI=11
Missing values in  DiabetesPedigreeFunction=0
Missing values in  Age=0


# Fixing Missing values:

* Glucose values from similar Age and Outcome values
* BloodPressure values from similar Age, Glucose and Outcome values average
* BMI values from similar Age and Glucose values average
* SkinThickness values from similar Age, BMI, Glucose and Outcome values average

In [7]:
def fix_miss_value(dataset, tar_col, need_col):
    #tar_col= str(name of column you want to fix missing values of)
    #need_col= list of columns for which values are to be compared
    l=[]
    pos_val=[]
    dataset[tar_col]= np.array(dataset[tar_col], dtype= 'float32')
    for i in range(len(dataset)):
        if dataset[tar_col][i]==0:
            for k in range(len(need_col)):
                for j in range(len(data)):
                    if dataset[need_col[k]][i]==dataset[need_col[k]][j]:
                        l.append(dataset[tar_col][j])
            avg= sum(l) / len(l)
            pos_val.append([i, avg])
            
    
    for i in range(len(pos_val)):
        k= pos_val[i][1] - int(pos_val[i][1])
        dataset[tar_col][pos_val[i][0]]= pos_val[i][1] 

In [8]:
# Replacing values as decided

fix_miss_value(data, "Glucose", ["Age", "Outcome"])

fix_miss_value(data, "BloodPressure", ["Age", "Glucose"])

fix_miss_value(data, "BMI", ["Age", "Glucose", "BloodPressure"])

fix_miss_value(data, "SkinThickness", ["BMI", "Age", "Glucose", "Outcome"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
#Not including Pregnencies
cols= ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]
k=0

for j in range(len(cols)-1):
    for i in range(len(data)):
        if data[cols[j+1]][i]==0:
            k=k+1
    print('Missing values in  ' + cols[j+1]+'='+ str(k))
    k=0

Missing values in  Glucose=0
Missing values in  BloodPressure=0
Missing values in  SkinThickness=0
Missing values in  Insulin=374
Missing values in  BMI=0
Missing values in  DiabetesPedigreeFunction=0
Missing values in  Age=0


## Scaling up Diabetes Pedigree Function entries

In [10]:
# Getting Diabetes Pedigree Function to same scale by multiplying it with a scaler(k= 100)
for i in range(len(data["DiabetesPedigreeFunction"])):
    data["DiabetesPedigreeFunction"][i]= data["DiabetesPedigreeFunction"][i]*100
data.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85.0,66.000000,29.000000,0,26.600000,35.1,31,0
1,1,89.0,66.000000,23.000000,94,28.100000,16.7,21,0
2,5,116.0,74.000000,19.473783,0,25.600000,20.1,30,0
3,10,115.0,66.230766,19.545033,0,35.299999,13.4,29,0
4,4,110.0,92.000000,19.550032,0,37.599998,19.1,30,0


# Feature Selection and splitting training-testing data

* Data contain more entries with 0 outcome(500 entries) in comparison to 1 outcome(268 entries). 
* And hence while training we will take 300 entries with 0 outcome and 250 entries with 1 outcome in training set.
* This leads to a split of 71:29 % for train:test  

In [11]:
train= pd.concat([data.iloc[0: 300], data.iloc[500: 750]]) # 300 values with 0 outcomes, and 250 values with 1 outcomes 
test=  pd.concat([data.iloc[300: 500], data.iloc[750: 768]]) # 300 values with 200 outcomes, and 18 values with 1 outcomes

train= train.set_index(i for i in range(len(train)))
test= test.set_index(i for i in range(len(test)))

In [12]:
# Note: we wont be including Insulint and Glucose readings as it requires skills and instruments to perform test which a normal user may not be have
X_train= train.drop(columns= ["Outcome", "Insulin", "Glucose"])
X_test= test.drop(columns= ["Outcome", "Insulin", "Glucose"])

Y_train= train["Outcome"]
Y_test= test["Outcome"]

In [13]:
# Testing and Validation Split
from sklearn.model_selection import train_test_split

X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.50)

In [14]:
# Converting 
X_train = np.array(X_train, dtype = 'float32')
Y_train = np.array(Y_train, dtype = 'float32')

X_test = np.array(X_test, dtype = 'float32')
Y_test = np.array(Y_test, dtype = 'float32')

X_val = np.array(X_val, dtype = 'float32')
Y_val = np.array(Y_val, dtype = 'float32')

In [15]:
torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(Y_train).type(torch.LongTensor) # data type is long

torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(Y_test).type(torch.LongTensor) # data type is long

torch_X_val = torch.from_numpy(X_val).type(torch.LongTensor)
torch_y_val = torch.from_numpy(Y_val).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)
val = torch.utils.data.TensorDataset(torch_X_val,torch_y_val)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = 10, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = 5, shuffle = False)
val_loader = torch.utils.data.DataLoader(val, batch_size = 5, shuffle = False)

# Defining Neural Network

In [16]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(in_features=6, out_features=40) # Linear layer 1 
        self.fc2 = nn.Linear(in_features=40, out_features=81) # Linear layer 2
        self.fc3 = nn.Linear(in_features=81, out_features=36) # Linear layer 3
        self.fc4 = nn.Linear(in_features=36, out_features=19) # Linear layer 4
        self.out = nn.Linear(in_features=19, out_features=2) # Linear layer 5 (output layer)
        
    def forward(self, t):
        # input layer
        t=t
        # (1) hidden conv layer
        t = self.fc1(t)
        t = F.relu(t)

        # (2) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (3) hidden linear layer
        t = self.fc3(t)
        t = F.relu(t)
        
        # (4) hidden linear layer
        t = self.fc4(t)
        t = F.relu(t)
        
        # (5) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

In [17]:
# Function for calculating correct predictions
def get_num_correct(preds, labels):
    return preds.argmax(dim= 1).eq(labels).sum().item()

In [18]:
network= Network()
network

Network(
  (fc1): Linear(in_features=6, out_features=40, bias=True)
  (fc2): Linear(in_features=40, out_features=81, bias=True)
  (fc3): Linear(in_features=81, out_features=36, bias=True)
  (fc4): Linear(in_features=36, out_features=19, bias=True)
  (out): Linear(in_features=19, out_features=2, bias=True)
)

# Training

In [25]:
network = Network()

optimizer = optim.SGD(network.parameters(), lr=0.001)

valid_loss_min = np.Inf

for epoch in range(1250):
    
    train_loss = 0
    train_correct = 0
    
    val_loss = 0
    val_correct = 0
    
    
    for batch in train_loader: # Get Batch
        paras, out = batch 

        preds = network(paras.float()) # Pass Batch
        loss = F.cross_entropy(preds, out.long()) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        train_loss += loss.item()
        train_correct += get_num_correct(preds, out)
        
        
        
    # For Validation
    
    for batch in val_loader: # Get Batch
        paras, out = batch 

        preds = network(paras.float()) # Pass Batch
        loss = F.cross_entropy(preds, out.long()) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        val_loss += loss.item()
        val_correct += get_num_correct(preds, out)

#     print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)
    print("Epoch", epoch, " ;Total Train correct:", train_correct, " ;Train loss:", train_loss)
    print("Total Validation correct:", val_correct ," ;Validation Loss:", val_loss)
    
    
    # save model if validation loss has decreased
    if val_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        val_loss))
        checkpoint = {'model': network,
              'state_dict': network.state_dict(),
              'optimizer' : optimizer.state_dict(),
              'train_loss': train_loss}

        torch.save(checkpoint, 'checkpoint.pth')
        valid_loss_min = val_loss
        
    print('\n')

Epoch 0  ;Total Train correct: 470  ;Train loss: 23.561714719980955
Total Validation correct: 63  ;Validation Loss: 17.155063211917877
Validation loss decreased (inf --> 17.155063).  Saving model ...


Epoch 1  ;Total Train correct: 482  ;Train loss: 20.940006010234356
Total Validation correct: 64  ;Validation Loss: 16.610895425081253
Validation loss decreased (17.155063 --> 16.610895).  Saving model ...


Epoch 2  ;Total Train correct: 478  ;Train loss: 21.427192263305187
Total Validation correct: 67  ;Validation Loss: 16.292696326971054
Validation loss decreased (16.610895 --> 16.292696).  Saving model ...


Epoch 3  ;Total Train correct: 474  ;Train loss: 21.862691070884466
Total Validation correct: 67  ;Validation Loss: 16.296582281589508


Epoch 4  ;Total Train correct: 474  ;Train loss: 22.333627838641405
Total Validation correct: 67  ;Validation Loss: 16.237347811460495
Validation loss decreased (16.292696 --> 16.237348).  Saving model ...


Epoch 5  ;Total Train correct: 471  ;

Epoch 42  ;Total Train correct: 448  ;Train loss: 25.28605354577303
Total Validation correct: 66  ;Validation Loss: 15.190860539674759
Validation loss decreased (15.202651 --> 15.190861).  Saving model ...


Epoch 43  ;Total Train correct: 449  ;Train loss: 25.23955962061882
Total Validation correct: 66  ;Validation Loss: 15.184149444103241
Validation loss decreased (15.190861 --> 15.184149).  Saving model ...


Epoch 44  ;Total Train correct: 450  ;Train loss: 25.183874670416117
Total Validation correct: 66  ;Validation Loss: 15.189861863851547


Epoch 45  ;Total Train correct: 450  ;Train loss: 25.142454463988543
Total Validation correct: 67  ;Validation Loss: 15.151324689388275
Validation loss decreased (15.184149 --> 15.151325).  Saving model ...


Epoch 46  ;Total Train correct: 450  ;Train loss: 25.048172879964113
Total Validation correct: 66  ;Validation Loss: 15.168529689311981


Epoch 47  ;Total Train correct: 450  ;Train loss: 25.01036311686039
Total Validation correct: 66  ;

Epoch 91  ;Total Train correct: 454  ;Train loss: 23.72417489439249
Total Validation correct: 68  ;Validation Loss: 14.907868891954422
Validation loss decreased (14.927012 --> 14.907869).  Saving model ...


Epoch 92  ;Total Train correct: 455  ;Train loss: 23.68083792924881
Total Validation correct: 67  ;Validation Loss: 14.929080128669739


Epoch 93  ;Total Train correct: 456  ;Train loss: 23.643877122551203
Total Validation correct: 67  ;Validation Loss: 14.90528815984726
Validation loss decreased (14.907869 --> 14.905288).  Saving model ...


Epoch 94  ;Total Train correct: 457  ;Train loss: 23.620949275791645
Total Validation correct: 67  ;Validation Loss: 14.919493556022644


Epoch 95  ;Total Train correct: 458  ;Train loss: 23.571504343301058
Total Validation correct: 67  ;Validation Loss: 14.952932596206665


Epoch 96  ;Total Train correct: 457  ;Train loss: 23.58255412429571
Total Validation correct: 67  ;Validation Loss: 14.915206789970398


Epoch 97  ;Total Train correct: 45

Epoch 139  ;Total Train correct: 456  ;Train loss: 23.416419811546803
Total Validation correct: 70  ;Validation Loss: 14.584253877401352
Validation loss decreased (14.600390 --> 14.584254).  Saving model ...


Epoch 140  ;Total Train correct: 456  ;Train loss: 23.38976164162159
Total Validation correct: 70  ;Validation Loss: 14.575179159641266
Validation loss decreased (14.584254 --> 14.575179).  Saving model ...


Epoch 141  ;Total Train correct: 456  ;Train loss: 23.356734309345484
Total Validation correct: 70  ;Validation Loss: 14.577549755573273


Epoch 142  ;Total Train correct: 456  ;Train loss: 23.38244415447116
Total Validation correct: 71  ;Validation Loss: 14.587228953838348


Epoch 143  ;Total Train correct: 456  ;Train loss: 23.340107310563326
Total Validation correct: 71  ;Validation Loss: 14.59669703245163


Epoch 144  ;Total Train correct: 456  ;Train loss: 23.365270987153053
Total Validation correct: 71  ;Validation Loss: 14.58700281381607


Epoch 145  ;Total Train corr

Epoch 185  ;Total Train correct: 454  ;Train loss: 23.10309987515211
Total Validation correct: 74  ;Validation Loss: 14.270674228668213


Epoch 186  ;Total Train correct: 454  ;Train loss: 23.100283335894346
Total Validation correct: 74  ;Validation Loss: 14.243684947490692
Validation loss decreased (14.257312 --> 14.243685).  Saving model ...


Epoch 187  ;Total Train correct: 455  ;Train loss: 23.092385038733482
Total Validation correct: 74  ;Validation Loss: 14.248592466115952


Epoch 188  ;Total Train correct: 455  ;Train loss: 23.085548050701618
Total Validation correct: 74  ;Validation Loss: 14.243350088596344
Validation loss decreased (14.243685 --> 14.243350).  Saving model ...


Epoch 189  ;Total Train correct: 455  ;Train loss: 23.091964453458786
Total Validation correct: 74  ;Validation Loss: 14.228776812553406
Validation loss decreased (14.243350 --> 14.228777).  Saving model ...


Epoch 190  ;Total Train correct: 455  ;Train loss: 23.083244644105434
Total Validation correc

...

Epoch 980  ;Total Train correct: 519  ;Train loss: 8.289587303996086
Total Validation correct: 96  ;Validation Loss: 6.389095555990934
Validation loss decreased (6.469210 --> 6.389096).  Saving model ...


Epoch 981  ;Total Train correct: 521  ;Train loss: 8.383389975409955
Total Validation correct: 97  ;Validation Loss: 6.446049364283681


Epoch 982  ;Total Train correct: 520  ;Train loss: 8.434131081216037
Total Validation correct: 96  ;Validation Loss: 6.417088208720088


Epoch 983  ;Total Train correct: 519  ;Train loss: 8.341452103573829
Total Validation correct: 95  ;Validation Loss: 6.468612466007471


Epoch 984  ;Total Train correct: 521  ;Train loss: 8.266867109574378
Total Validation correct: 96  ;Validation Loss: 6.436086103320122


Epoch 985  ;Total Train correct: 520  ;Train loss: 8.29698536079377
Total Validation correct: 97  ;Validation Loss: 6.3741227351129055
Validation loss decreased (6.389096 --> 6.374123).  Saving model ...


Epoch 986  ;Total Train correct: 521  ;T

Epoch 1032  ;Total Train correct: 521  ;Train loss: 7.7192793970461935
Total Validation correct: 96  ;Validation Loss: 5.747017468325794
Validation loss decreased (5.769829 --> 5.747017).  Saving model ...


Epoch 1033  ;Total Train correct: 520  ;Train loss: 7.80014885799028
Total Validation correct: 97  ;Validation Loss: 5.799023265019059


Epoch 1034  ;Total Train correct: 520  ;Train loss: 7.748251487035304
Total Validation correct: 97  ;Validation Loss: 5.834713394753635


Epoch 1035  ;Total Train correct: 522  ;Train loss: 7.7642569919116795
Total Validation correct: 97  ;Validation Loss: 5.822543159127235


Epoch 1036  ;Total Train correct: 521  ;Train loss: 7.748677532188594
Total Validation correct: 98  ;Validation Loss: 5.737690255045891
Validation loss decreased (5.747017 --> 5.737690).  Saving model ...


Epoch 1037  ;Total Train correct: 521  ;Train loss: 7.759867822285742
Total Validation correct: 97  ;Validation Loss: 5.8237776551395655


Epoch 1038  ;Total Train correct

Epoch 1086  ;Total Train correct: 520  ;Train loss: 7.137263581156731
Total Validation correct: 99  ;Validation Loss: 5.100067000836134


Epoch 1087  ;Total Train correct: 521  ;Train loss: 7.0604217909276485
Total Validation correct: 100  ;Validation Loss: 5.142722359858453


Epoch 1088  ;Total Train correct: 520  ;Train loss: 7.069850399857387
Total Validation correct: 99  ;Validation Loss: 5.045187037438154
Validation loss decreased (5.085712 --> 5.045187).  Saving model ...


Epoch 1089  ;Total Train correct: 522  ;Train loss: 7.12096312455833
Total Validation correct: 98  ;Validation Loss: 5.085913052782416


Epoch 1090  ;Total Train correct: 521  ;Train loss: 6.969940724782646
Total Validation correct: 99  ;Validation Loss: 5.118790376931429


Epoch 1091  ;Total Train correct: 520  ;Train loss: 6.995644165202975
Total Validation correct: 99  ;Validation Loss: 5.054009979590774


Epoch 1092  ;Total Train correct: 521  ;Train loss: 6.975953478598967
Total Validation correct: 100  ;



Epoch 1141  ;Total Train correct: 525  ;Train loss: 6.0516641666181386
Total Validation correct: 101  ;Validation Loss: 4.6663967510685325


Epoch 1142  ;Total Train correct: 522  ;Train loss: 6.195504103205167
Total Validation correct: 101  ;Validation Loss: 4.621475073508918
Validation loss decreased (4.628724 --> 4.621475).  Saving model ...


Epoch 1143  ;Total Train correct: 524  ;Train loss: 6.0980546668870375
Total Validation correct: 100  ;Validation Loss: 4.60965683311224
Validation loss decreased (4.621475 --> 4.609657).  Saving model ...


Epoch 1144  ;Total Train correct: 523  ;Train loss: 6.038037302671
Total Validation correct: 100  ;Validation Loss: 4.568554216995835
Validation loss decreased (4.609657 --> 4.568554).  Saving model ...


Epoch 1145  ;Total Train correct: 525  ;Train loss: 6.034317700075917
Total Validation correct: 101  ;Validation Loss: 4.615409009158611


Epoch 1146  ;Total Train correct: 521  ;Train loss: 6.169619578402489
Total Validation correct: 1

Epoch 1193  ;Total Train correct: 525  ;Train loss: 5.271245332551189
Total Validation correct: 101  ;Validation Loss: 4.144899820908904


Epoch 1194  ;Total Train correct: 529  ;Train loss: 5.245904063107446
Total Validation correct: 101  ;Validation Loss: 4.056864550337195


Epoch 1195  ;Total Train correct: 529  ;Train loss: 5.172479222295806
Total Validation correct: 101  ;Validation Loss: 4.032330833375454


Epoch 1196  ;Total Train correct: 527  ;Train loss: 5.239259417634457
Total Validation correct: 101  ;Validation Loss: 4.059458080679178


Epoch 1197  ;Total Train correct: 529  ;Train loss: 5.099326286232099
Total Validation correct: 102  ;Validation Loss: 4.019084611907601


Epoch 1198  ;Total Train correct: 529  ;Train loss: 5.029658391722478
Total Validation correct: 101  ;Validation Loss: 4.110411571338773


Epoch 1199  ;Total Train correct: 528  ;Train loss: 5.174598450423218
Total Validation correct: 101  ;Validation Loss: 3.9536720272153616
Validation loss decreased (4

# Load the best model(with minimum validation loss):

In [26]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    train_loss= checkpoint['train_loss']
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model, train_loss

In [27]:
network_best, train_loss = load_checkpoint('checkpoint.pth')
network_best= network_best
train_loss= train_loss
print(network_best)

Network(
  (fc1): Linear(in_features=6, out_features=40, bias=True)
  (fc2): Linear(in_features=40, out_features=81, bias=True)
  (fc3): Linear(in_features=81, out_features=36, bias=True)
  (fc4): Linear(in_features=36, out_features=19, bias=True)
  (out): Linear(in_features=19, out_features=2, bias=True)
)


# Checking Accuracy

In [28]:
accuracy_train= (train_correct/len(train))*100
print('Train Accuracy=' + str(accuracy_train) + '%')

Train Accuracy=96.36363636363636%


# Checking on Testing Test

In [29]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

network_best.eval() # prep model for evaluation

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = network_best(data.float())
    
    # calculate the loss
    loss = F.cross_entropy(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    
    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        continue

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.658432

Test Accuracy of     0: 90% (90/99)
Test Accuracy of     1: 30% ( 3/10)

Test Accuracy (Overall): 85% (93/109)


In [30]:
# Save the model as a pickle in a file 
joblib.dump(network_best, 'FeCare.pkl') 

['FeCare.pkl']